In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras as k
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import skimage.morphology as morp
from skimage.filters import rank

%matplotlib inline

AttributeError: module 'pandas' has no attribute 'core'

In [ ]:
train = pd.read_csv(""D:/mapmyindia-master/mapmyindia-master/data/train.csv")
train_labels = train["Label"].values
train = train.sort_values("Img_Name")
train

In [ ]:
# Load all images. 

import os
import skimage.data
def load_data(data_dir):
    images = []
    
    file_names = sorted([os.path.join(data_dir, f)
                  for f in os.listdir(data_dir) if f.endswith(".jpg")])
    
    for f in file_names:
        images.append(skimage.data.imread(f, as_gray=True))
    
    return images

images = load_data('"D:/mapmyindia-master/mapmyindia-master/data/train/cropped')
print(len(images))

In [ ]:
#explore shapes
for image in images[:3]:
    print('image.shape[before]', image.shape)

In [ ]:
#transform all images to 250x250
import skimage.transform
images250 = [skimage.transform.resize(image, (250, 250)) for image in images]

In [ ]:
#confirm transformation
for image in images250[:3]:
    print('image.shape[after]', image.shape)
    

In [ ]:
from PIL import Image

for image in images250[:3]:
    image = (image * 255).astype(np.uint8)
    im = Image.fromarray(image)
    plt.figure()
    plt.imshow(im)
    plt.show()



In [ ]:
#Load Labels 
data = pd.read_csv("../data/train.csv")
labels = data["Label"]

labels.head()

In [ ]:
#categorize labels..

import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

y_train_labels = np.array(labels)
X_train_data = np.array(images250)

In [ ]:
len(y_train_labels)

In [ ]:
num_categories = 6

## hack (FIXME).... train & label len do not match, so delete a row from labels.
y_train_labels = np.delete(y_train_labels, (15), axis=0)

#encode labels
label_encoder = LabelEncoder()
y_labels_encoded = label_encoder.fit_transform(y_train_labels)

y_labels_categorized = to_categorical(y_labels_encoded, num_categories)

print('len(X_train_data)', len(X_train_data))
print('len(y_train_labels)', len(y_train_labels))
print('len(y_labels_categorized)', len(y_labels_categorized))

In [ ]:
input_shape = (250, 250, 3)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, data_format='channels_first'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_categories, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

model.summary()

model.fit(X_train_data, y_labels_categorized, nb_epoch=10, batch_size=100)